In [1]:
import talib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import datetime
import mplfinance as mpf

from Trader import Trader
from DivergenceDetector import DivergenceDetector

In [31]:
data_paths = [
    'data/2022/january/ethusdt.15min.1january2022-1february2022.csv',
    'data/2022/february/ethusdt.15min.1-28feb2022.csv',
    'data/2022/march/ethusdt.15min.1march2022-21march2022.csv'
]

df = pd.concat([pd.read_csv(d, index_col='Open time', usecols=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume']) for d in data_paths])
df = df.drop_duplicates(keep='first')
df = df.dropna()

df.index = pd.to_datetime(df.index)
df = df.sort_values('Open time')

In [98]:
Data = np.c_[df.drop('Volume', axis=1).to_numpy(), talib.RSI(df.Close, 14).to_numpy(), talib.ATR(df.High, df.Low, df.Close, 14)]
Data = Data[~np.isnan(Data[:, 4:6]).any(axis=1)]

In [101]:
df

,Open,High,Low,Close,Volume
Open time,,,,,
2022-01-01 00:00:00,3676.01,3705.90,3676.01,3691.50,15789.481
2022-01-01 00:15:00,3691.50,3698.98,3684.66,3696.81,5576.187
2022-01-01 00:30:00,3696.81,3716.61,3693.44,3712.52,10722.556
2022-01-01 00:45:00,3712.51,3730.00,3709.47,3721.67,13747.696
2022-01-01 01:00:00,3721.67,3727.04,3712.65,3726.16,10340.489
...,...,...,...,...,...
2022-03-20 23:00:00,2876.00,2876.24,2868.96,2874.67,6626.714
2022-03-20 23:15:00,2874.67,2874.67,2854.78,2855.35,14717.313
2022-03-20 23:30:00,2855.36,2859.87,2850.38,2856.22,10316.699


In [107]:
def rnr(entry, stoploss, rr):
    return (entry - sl) * rr

In [122]:
entry, sl, position, tp

(3720.22, 3700.6984844555122, 1, 3749.502273316731)

In [148]:
trader = Trader(leverage=20)
trader.reset()

lower_barrier = 30
upper_barrier = 70
width = 10

rr = 1.5
entry = None
position = None
sl = None

# Bullish Divergence
for i in range(len(Data)):
    if position != None:
        if (Data[i, 3] <= sl) or (Data[i, 3] >= tp):
            trader.close_pos(Data[i, 3])
            print(entry, sl, tp, Data[i, 3])
            entry = None
            position = None
            sl = None
            tp = None
            
    if Data[i, 4] < lower_barrier:
        for a in range(i + 1, i + width):
            if Data[a, 4] > lower_barrier:
                for r in range(a + 1, a + width):
                    if Data[r, 4] < lower_barrier and Data[r, 4] > Data[i, 4] and Data[r, 3] < Data[i, 3]:
                        for s in range(r + 1, r + width): 
                            if Data[s, 4] > lower_barrier:
                                # long
                                if position == None:
                                    trader.open_pos(Data[s+1, 3], 'long')
                                    entry = Data[s+1, 3]
                                    position = 1
                                    sl = entry - Data[s+1, 5]
                                    tp = entry + rnr(entry, sl, rr)
                                    break
                            else:
                                continue
                    else:
                        continue
            else:
                continue
    else:
        continue


3720.22 3700.6984844555122 3749.502273316731 3765.53
3532.65 3487.6557866673716 3600.141319998943 3721.98
3207.04 3177.3418433583165 3251.587234962525 3165.87
3017.56 2983.2833544048203 3068.9749683927694 3160.97
3008.51 2973.9859911178537 3060.29601332322 3114.5
3008.51 2973.9859911178537 3060.29601332322 3116.0
3008.51 2973.9859911178537 3060.29601332322 3083.6
3008.51 2973.9859911178537 3060.29601332322 3061.68
3008.51 2973.9859911178537 3060.29601332322 3065.59
3252.12 3237.852610411635 3273.521084382547 3279.96
3070.79 3055.0238486484527 3094.439227027321 3096.48
3070.79 3055.0238486484527 3094.439227027321 3103.4
2563.35 2511.250854259585 2641.498718610622 2495.0
2455.34 2404.0171649382255 2532.324252592662 2388.1
2400.9 2377.909040058946 2435.386439911581 2437.35
2186.55 2147.196530206059 2245.580204690912 2347.67
2186.55 2147.196530206059 2245.580204690912 2333.59
2186.55 2147.196530206059 2245.580204690912 2334.48
2186.55 2147.196530206059 2245.580204690912 2272.15
2186.55 214

In [143]:
pd.DataFrame(Data)

,0,1,2,3,4,5
0,3727.51,3730.76,3725.44,3729.45,78.859316,12.365714
1,3729.45,3729.98,3720.56,3722.07,70.354985,12.155306
2,3722.07,3726.63,3710.51,3711.74,60.517252,12.438499
3,3711.95,3718.57,3710.46,3715.63,62.636038,12.129320
4,3715.62,3717.13,3707.40,3714.28,61.404497,11.957940
...,...,...,...,...,...,...
7566,2876.00,2876.24,2868.96,2874.67,52.607554,15.265240
7567,2874.67,2874.67,2854.78,2855.35,44.072726,15.595580
7568,2855.36,2859.87,2850.38,2856.22,44.509306,15.159467
7569,2856.22,2863.04,2853.52,2860.47,46.698256,14.756648


In [144]:
trader.account_size

581.6089023841544

In [145]:
pd.DataFrame(trader.history)

,entry_price,position,entry_margin,entry_fee,number_contract,exit_price,profit,percentage,exit_fee,account_size
0,3720.22,long,10.000000,0.080000,0.053760,3765.53,2.435877,0.243588,0.080000,102.275877
1,3532.65,long,10.227588,0.081821,0.057903,3721.98,10.962814,1.071887,0.081821,113.075050
2,3207.04,long,11.307505,0.090460,0.070517,3165.87,-2.903175,-0.256748,0.090460,109.990954
3,3017.56,long,10.999095,0.087993,0.072901,3160.97,10.454674,0.950503,0.087993,120.269643
4,3008.51,long,12.026964,0.096216,0.079953,3114.50,8.474214,0.704601,0.096216,128.551426
5,3008.51,long,12.855143,0.102841,0.085459,3116.00,9.185938,0.714573,0.102841,137.531681
6,3008.51,long,13.753168,0.110025,0.091428,3083.60,6.865361,0.499184,0.110025,144.176991
7,3008.51,long,14.417699,0.115342,0.095846,3061.68,5.096138,0.353464,0.115342,149.042446
8,3008.51,long,14.904245,0.119234,0.099081,3065.59,5.655519,0.379457,0.119234,154.459497
9,3252.12,long,15.445950,0.123568,0.094990,3279.96,2.644523,0.171211,0.123568,156.856885


In [139]:
trader = Trader(leverage=5)
trader.reset()

lower_barrier = 30
upper_barrier = 70
width = 10

rr = 1.5
entry = None
position = None
sl = None

# Bearish Divergence
for i in range(len(Data)):
    if position != None:
        if (Data[i, 3] >= sl) or (Data[i, 3] <= tp):
            trader.close_pos(Data[i, 3])
            entry = None
            position = None
            sl = None
            tp = None

    try:
        if Data[i, 4] > upper_barrier:
            for a in range(i + 1, i + width): 
                if Data[a, 4] < upper_barrier:
                    for r in range(a + 1, a + width):
                        if Data[r, 4] > upper_barrier and Data[r, 4] < Data[i, 4] and Data[r, 3] > Data[i, 3]:
                            for s in range(r + 1, r + width):
                                if Data[s, 4] < upper_barrier:
                                    # short
                                    if position == None:
                                        trader.open_pos(Data[s+1, 3], 'short')
                                        entry = Data[s+1, 3]
                                        position = -1
                                        sl = entry - Data[s+1, 5]
                                        tp = entry + rnr(entry, sl, rr)
                                        break
                                else:
                                    continue
                        else:
                            continue
                else:
                    continue
        else:
            continue
    except IndexError:
        pass

In [140]:
trader.account_size

111.0803034202195

In [141]:
pd.DataFrame(trader.history)

,entry_price,position,entry_margin,entry_fee,number_contract,exit_price,profit,percentage,exit_fee,account_size
0,3735.39,short,10.000000,0.020000,0.013385,3722.07,0.178295,0.017829,0.020000,100.138295
1,3838.94,short,10.013829,0.020028,0.013042,3824.05,0.194202,0.019393,0.020028,100.292441
2,3838.94,short,10.029244,0.020058,0.013063,3818.53,0.266606,0.026583,0.020058,100.518930
3,3838.94,short,10.051893,0.020104,0.013092,3831.09,0.102772,0.010224,0.020104,100.581495
4,3849.02,short,10.058150,0.020116,0.013066,3836.19,0.167635,0.016667,0.020116,100.708897
...,...,...,...,...,...,...,...,...,...,...
76,2947.29,short,11.096237,0.022192,0.018824,2934.40,0.242647,0.021868,0.022192,111.160632
77,2947.29,short,11.116063,0.022232,0.018858,2932.55,0.277969,0.025006,0.022232,111.394136
78,2947.29,short,11.139414,0.022279,0.018898,2939.64,0.144568,0.012978,0.022279,111.494146
79,2947.29,short,11.149415,0.022299,0.018915,2964.98,-0.334601,-0.030011,0.022299,111.114947
